In [ ]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

import pandas as pd
pd.core.common.is_list_like = pd.api.types.is_list_like
try:
    import empyrical as emp
except:
    emp = None
import tushare as ts
import time
import random

from common.log import *
from common.config import Config
from spider.spider_nasdaq import Spider_nasdaq
from spider.spider_coinmarketcap import Spider_coinmarketcap
from spider.spider_okex import Spider_okex

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

from pandas_highcharts.core import serialize
from pandas_highcharts.display import display_charts

import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [15, 10]

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:70% !important; }</style>"))

CONF = Config('../conf/secret.yaml').data[0]
ts_token = CONF['TUSHARE']['TOKEN']
ts.set_token(ts_token)
pro = ts.pro_api()

CONF = Config().data[0]
MONGODB = CONF['MONGODB']
NASDAQ = CONF['NASDAQ']
NASDAQ100 = CONF['NASDAQ100']
CRYPTOCURRENCY = CONF['CRYPTOCURRENCY']
CRYPTOCURRENCY = list(CRYPTOCURRENCY.keys())
CRYPTOCURRENCYSYMBOL = CONF['CRYPTOCURRENCYSYMBOL']

MONITOR_TARGET = {
    '399300': {'MA': [17], 'ROLLINGMAX': [], 'ROLLINGMIN': [], 'N': 10, 'NAME': '沪深300(SZ:399300)'},
    '000905': {'MA': [17], 'ROLLINGMAX': [], 'ROLLINGMIN': [], 'N': 10, 'NAME': '中证500(SH:000905)'},
    '399006': {'MA': [17], 'ROLLINGMAX': [], 'ROLLINGMIN': [], 'N': 10, 'NAME': '创业板指(SZ:399006)'},
    'BTC': {'MA': [30], 'ROLLINGMAX': [], 'ROLLINGMIN': [], 'N': 2, 'NAME': 'BITCOIN'},
    'EOS': {'MA': [30], 'ROLLINGMAX': [], 'ROLLINGMIN': [], 'N': 2, 'NAME': 'EOS'},
    'ETH': {'MA': [30], 'ROLLINGMAX': [], 'ROLLINGMIN': [], 'N': 2, 'NAME': 'ETHEREUM'},
    'XRP': {'MA': [30], 'ROLLINGMAX': [], 'ROLLINGMIN': [], 'N': 2, 'NAME': 'RIPPLE'},
    'LTC': {'MA': [30], 'ROLLINGMAX': [], 'ROLLINGMIN': [], 'N': 2, 'NAME': 'LITECOIN'},
}

pd

In [ ]:
%%time

IS_DOWNLOAD_DATA = True

spider_n = Spider_nasdaq()
spider_c = Spider_coinmarketcap()
spider_o = Spider_okex()

for symbol in MONITOR_TARGET:
    if not IS_DOWNLOAD_DATA:
        break
        
    print(symbol)

    # 指数
    if symbol in ['399300', '000905', '000016', '399006']:
        df = ts.get_k_data(code=symbol, index=True, start='2005-01-01', end=time.strftime('%Y-%m-%d'))
        datafile = '../database/market/%s.csv' % symbol
        df.to_csv(datafile, index=False, encoding='utf-8')
        
    # CRYPTOCURRENCY
    elif symbol in CRYPTOCURRENCYSYMBOL:
        datafile = spider_o.get_coin_data(symbol)
        print(datafile)
        
    # ETF
    else:
        df = ts.get_k_data(code=symbol, ktype='D', autype='qfq', index=False, start='2005-01-01', end=time.strftime('%Y-%m-%d'))
        datafile = '../database/market/%s.csv' % symbol
        df.to_csv(datafile, index=False, encoding='utf-8')
        time.sleep(1)

    time.sleep(1)

In [ ]:
# %run OKAPI.ipynb

In [ ]:
%%time

stock_df_dict = {}

for symbol in MONITOR_TARGET:
    stock_data_file = '../database/market/%s.csv' % symbol
    if symbol in CRYPTOCURRENCYSYMBOL:
        stock_data_file = '../database/market/%s_OKEX.csv' % symbol
    try:
        stock_df = pd.read_csv(stock_data_file)
    except:
        print(symbol)
        continue

    # 筛选字段
    stock_df = stock_df.loc[:, ['date', 'close']]

    # 抛弃空值异常值
    stock_df.dropna(axis=0, how='any', inplace=True)

    # 格式化日期
    # 445 ms ± 17.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
    stock_df = stock_df.assign(date=pd.to_datetime(stock_df['date']))  # need .index.to_period('D')

    # 用日期作索引，日期升序排序
    # 95.1 µs ± 1.58 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
    if symbol in NASDAQ100 or symbol in CRYPTOCURRENCYSYMBOL:
#         stock_df = stock_df[1:]
        stock_df = stock_df[::-1]
    stock_df.set_index(['date'], inplace=True)
    stock_df.index = stock_df.index.to_period('D')

    # 计算每天涨跌幅
    N = MONITOR_TARGET[symbol]['N']
    stock_df['N_sht'] = stock_df.close.shift(N-1)
    stock_df['N_chg'] = (stock_df.close.shift(0) - stock_df.close.shift(N-1)) / stock_df.close.shift(N-1)

    for MA_PARAM in MONITOR_TARGET[symbol]['MA']:
        stock_df['MA_%d' % MA_PARAM] = stock_df['close'].rolling(MA_PARAM).mean()
    
    for ROLLINGMAX_PARAM in MONITOR_TARGET[symbol]['ROLLINGMAX']:
        stock_df['ROLLINGMAX_%d' % ROLLINGMAX_PARAM] = stock_df['close'].rolling(ROLLINGMAX_PARAM).max()
        
    for ROLLINGMIN_PARAM in MONITOR_TARGET[symbol]['ROLLINGMIN']:
        stock_df['ROLLINGMIN_%d' % ROLLINGMIN_PARAM] = stock_df['close'].rolling(ROLLINGMIN_PARAM).min()
    
    # 减少数据
    stock_df.dropna(how='any', inplace=True)
    
    stock_df_dict[symbol] = stock_df

In [ ]:
%store MONITOR_TARGET
%store stock_df_dict

In [ ]:
# for symbol in MONITOR_TARGET:
#     symbol
#     stock_df_dict[symbol].tail(2)

In [ ]:
monitor_df = pd.DataFrame(columns=['SYMBOL', 'DATE', 'CLOSE'])
#  'BUY', 'BUYDIFF', 'SELL', 'SELLDIFF'
for symbol in MONITOR_TARGET:
    df = stock_df_dict[symbol].iloc[-30:].copy()
    
    df.reset_index(drop=False, inplace=True)
    # df = df.astype(dtype={'date': 'datetime64[ns]'})
    df['date'] = df['date'].apply(lambda x: x.to_timestamp().to_datetime64())
    df.set_index('date', inplace=True)
    
    today_market = df.iloc[-1]
    now_point = today_market.close
    for col in df.columns:
        if 'ROLLINGMAX' in col:
            buy_point = today_market[col]
            buy_diff = (buy_point - now_point) / now_point * 100
        elif 'ROLLINGMIN' in col:
            sell_point = today_market[col]
            sell_diff = (now_point - sell_point) / now_point * 100
        elif 'MA' in col:
            ma = today_market[col]
            ma_diff = (now_point - ma) / now_point * 100
        elif 'N_chg' in col:
            n_chg = today_market[col] * 100
    
    monitor_df = monitor_df.append({
        'SYMBOL': symbol, 
        'DATE': today_market.name.date(), 
        'CLOSE': now_point, 
#         'BUY': buy_point, 
#         'BUYDIFF': '+%.2f%%' % buy_diff, 
#         'SELL': sell_point, 
#         'SELLDIFF': '-%.2f%%' % sell_diff,
        'MA': ma,
        'MADIFF': '%.2f%%' % ma_diff,
        'N_sht': today_market.N_sht, 
        'N_chg': '%.2f%%' % n_chg,
    }, ignore_index=True)
    
    title = symbol
    title = '%s, %s, now=%.2f, N_chg=%.2f(%.2f%%), MADIFF=%.2f(%.2f%%)' % \
        (symbol, today_market.name.date(), now_point, today_market.N_sht, n_chg, ma, ma_diff)
    
#     df.drop(columns=['N_chg'], inplace=True)
#     ax = df.plot(kind='line', title=title, linewidth=0.9, grid=True, figsize=(19, 5))
#     ax.yaxis.tick_right()
    
#     display_charts(df, chart_type='stock', kind='line', title=title, figsize=(1000, 600))


monitor_df

In [ ]:
# df = ts.get_realtime_quotes(['sh', 'sz', 'hs300', 'sz50', 'sh000905', 'zxb', 'cyb'])
# df = ts.get_realtime_quotes(['hs300', 'sh000905', 'cyb'])
# df = ts.get_realtime_quotes(list(MONITOR_TARGET.keys()))
# df
# list(MONITOR_TARGET.keys())

# ts
pro
df = pro.index_daily(ts_code='399300.SZ', start_date='20190601', end_date='20190611')
df

df = ts.get_k_data(code='399300', index=True, start='2019-06-01', end=time.strftime('%Y-%m-%d'))
df